## Schelling's  Model of Segregation
Adapted from Frank McCown and Stanford's Nifty Assignments repository.

### Overview
Racial segregation has always been a pernicious social problem in the United States. Although much effort has been extended to desegregate our schools, churches, and neighborhoods, the [US continues to remain segregated](https://tcf.org/content/commentary/racial-segregation-is-still-a-problem/) by race and economic lines. Why is segregation such a difficult problem to eradicate?

In 1971, the American economist [Thomas Schelling](https://en.wikipedia.org/wiki/Thomas_Schelling) created an agent-based model that might help explain why segregation is so difficult to combat. His model of segregation showed that even when individuals (or "agents") didn't mind being surrounded or living by agents of a different race, they would still choose to segregate themselves from other agents over time! Although the model is quite simple, it gives a fascinating look at how individuals might self-segregate, even when they have no explicit desire to do so.

### How the Model Works

Schelling's model will now be explained with some minor changes. Suppose there are two types of agents: X and O. The two types of agents might represent different races, ethnicity, economic status, etc. Two populations of the two agent types are initially placed into random locations of a neighborhood represented by a grid. After placing all the agents in the grid, each cell is either occupied by an agent or is empty as shown below. 

![alt text](http://www.cs.columbia.edu/~cannon/images/grid1.png "Agents placed randomly")

Now we must determine if each agent is satisfied with its current location. A satisfied agent is one that is surrounded by at least a fraction *t* of agents that are like itself. This threshold *t* is one that will apply to all agents in the model, even though in reality everyone might have a different threshold they are satisfied with. Note that the higher the threshold, the higher the likelihood the agents will not be satisfied with their current location.

For example, if *t = .3*, agent X is satisfied if at least 30% of its neighbors are also X. If fewer than 30% are X, then the agent is not satisfied, and it will want to change its location in the grid. For the remainder of this explanation, let's assume a threshold *t* of .3. This means every agent is fine with being in the minority as long as there are at least 30% of similar agents in adjacent cells.

The picture below shows a satisfied agent because 50% of X's neighbors are also X (.5 > t). 

![alt text](http://www.cs.columbia.edu/~cannon/images/grid2.png "Satisfied agent") 

In the picture below, agent X is not satisfied because only 25% of its neighbors are X (.25 < t). Notice that in this example empty cells are not counted when calculating similarity.

![alt text](http://www.cs.columbia.edu/~cannon/images/grid3.png "Dissatisfied agent")

When an agent is not satisfied, it can be moved to any vacant location in the grid. Any algorithm can be used to choose this new location. For example, a randomly selected cell may be chosen, or the agent could move to the nearest available location.

In the image below, all dissatisfied agents have an asterisk next to them. 


![alt text](http://www.cs.columbia.edu/~cannon/images/grid4.png "Dissatisfied agents with asterisk") 

The next image shows the new configuration after all the dissatisfied agents have been moved to unoccupied cells at random. Note that the new configuration may cause some agents which were previously satisfied to become dissatisfied! 

![alt text](http://www.cs.columbia.edu/~cannon/images/grid5.png "New configuration") 

All dissatisfied agents must be moved in the same round. After the round is complete, a new round begins, and dissatisfied agents are once again moved to new locations in the grid. These rounds continue until all agents in the neighborhood are satisfied with their location.

Consider the following parameters for the Schelling model:
* grid size: tuple of integers (n,m)
* fraction of population 1: float between 0 and 1.
* fraction of vacant sites in the grid: float between 0 and 1.
* tolerance: float between 0 and 1

`start(grid_size,pop,vacancy)` creates and returns a 2-D numpy array of the appropriate size consisting of integers where each element of the array is assigned as follows:
* part of first demographic:  1
* part of second demographic: -1
* vacant: 0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# NOTE: This function assumes that population and vacancy sites are proper fractions of the grid size.

def start(grid_size, pop, vacancy):

    # Set variable values for the first and second demographics, demographic_1 demographic_2
    demographic_1 = 1
    demographic_2 = -1
    
    # Calcultate the array length using grid_size, array_length
    array_length = grid_size[0] * grid_size[1]
    
    # Create a NumPy array of zeroes with length array_length, pop_array
    pop_array = np.zeros(array_length)
    
    # Calculate the number of agents in the first demographic by multiplying array_length by pop, demographic_1_pop
    demographic_1_pop = array_length * pop
    
    # Calculate the number of vacancy sites by multiply array_length by vacancy, vacancy_sites
    vacancy_sites = array_length * vacancy
    
    # Calculate the number of agents in the second demographic from the remaining elements in array_length, demographic_2_pop
    demographic_2_pop = array_length * (1 - pop) - vacancy_sites
    
    # Fill pop_array with 1 for each of the first demographic population
    for i in range(int(demographic_1_pop)):
        pop_array[i] = 1
    
    # Fill pop_array with -1 for each of the second demographic population
    for i in range(1, int(demographic_2_pop + 1)):
        pop_array[-i] = -1
        
    # Randomize pop_array
    np.random.shuffle(pop_array)
    
    # Change pop_array to the grid_size dimensions, schelling_grid
    arr = np.reshape(pop_array, grid_size)
    
    
    return arr

In [ ]:
# Test the start function
grid = start((4, 5), .5, .2)
print(grid)


plt.matshow(grid, cmap='jet')
plt.show()


`next_round(arr,tolerance)` takes as input an *n by m* array and returns a new 2-D array one step in the future where all dissatisfied agents have been moved. Remember, since there may be more dissatisfied agents than vacancies in the grid, they need to move one at a time, this way each time an agent is randomly assigned to a vacant site, a vacancy at a new location is created.

In [ ]:
def next_round(arr, tolerance):

    # Create variables for the number of rows and columns in arr, arr_row arr_columns
    arr_rows = arr.shape[0]

    arr_columns = arr.shape[1]


    # Create a list containing the coordinates of dissatisfied agents as tuples, dissatisfied_list
    dissatisfied_list = []

    # Create a lambda function which creates a list containing the coordinates of each neighbor, find_neighbors
    # This is generalized to any grid size
    find_neighbors = lambda row, column : [(neighbor_row, neighbor_column) for neighbor_row in range(row-1, row+2)
                                           for neighbor_column in range(column-1, column+2)
                                           if (-1 < row < arr_rows and
                                           -1 < column < arr_columns and
                                           (row != neighbor_row or column != neighbor_column) and
                                           (0 <= neighbor_row < arr_rows) and
                                           (0 <= neighbor_column < arr_columns))]


    # Iterrate through each row and column of the grid
    for agent_row in range(0, arr_rows):
        for agent_column in range(0, arr_columns):

            # Create a variable to count the number of neighbors of the same type as the agent, same_count
            same_count = 0

            # Ignore vacancies
            if arr[agent_row, agent_column] != 0:

                # Check the values of the neighbors for each coordinate of the grid
                for neighbor in find_neighbors(agent_row, agent_column):

                    # If the neighbor is the same type as the agent, increase same_count by 1
                    if arr[neighbor] == arr[agent_row, agent_column]:
                        same_count += 1

                # Check if the percentage of like neighbors is below the tolerance threshold
                if same_count / len(find_neighbors(agent_row, agent_column)) < tolerance:
                    # Apend the agent coordinates to dissatisfied_list if they are below the tolerance threshold
                    dissatisfied_list.append((agent_row, agent_column))


    # Move the dissatisfied agents to vacant spaces in the grid
    for agent_coordinates in dissatisfied_list:
        # Create random coordinates within the arr grid, random_row random_column
        random_row = np.random.randint(0, arr_rows)
        random_column = np.random.randint(0, arr_columns)

        # The random coordinates need to have a value of zero (vacant)
        while arr[random_row, random_column] != 0:
            # Re-randomize the coordinates if they are not vacant
            random_row = np.random.randint(0, arr_rows)
            random_column = np.random.randint(0, arr_columns)

        # Replace the vacant space with the value of the agent
        arr[random_row, random_column] = arr[agent_coordinates]

        # Set the starting agent coordinates to zero (vacant)
        arr[agent_coordinates] = 0

    return arr


In [ ]:
# Test the next_round function
arr = start((5, 5), 0.20, 0.40)
tolerance = 0.20

print(arr)
print('')
print(next_round(arr, tolerance))


In [ ]:
# Visual example of next_round
a = np.ones((4, 5))
a[2:4] = -1
a[1:3, 0:2] = 0
plt.matshow(a, cmap='jet')
plt.show()

b = next_round(a, .6)
plt.matshow(b, cmap='jet')
plt.show()


`make_list(n)` returns a list of *n* numpy arrays where the first array is created using `start` and the second using `next_round` on the first array and the third using `next_round` on the second, and so on. 

In [ ]:
def make_list(n, grid_size, pop, vacancy, tolerance):

    # Create a list of numpy arrays, schelling_grid_list
    schelling_grid_list = []

    # Create a 2-D numpy array using the start function, arr
    arr = start(grid_size, pop, vacancy)

    # Append the array to schelling_grid_list
    schelling_grid_list.append(arr)

    next_arr = np.copy(arr)

    # Repeat the next_round function to create the remaining number of arrays
    for i in range(1, n):

        # Append the array to schelling_grid_list
        schelling_grid_list.append(next_arr)

        # Move the dissatisfied agents
        next_arr = np.copy(next_round(next_arr, tolerance))

    return schelling_grid_list


In [ ]:
# Test the make_list function
test = make_list(4, (5, 5), .20, .40, .20)

print(test)

for mat in test:
    plt.matshow(mat, cmap='jet')
    plt.show()


__Test Code__


In [ ]:
n = 100
grid_size = (30, 30)
pop = 0.5
vacancy = 0.1
tolerance = 0.3

make_list(n, grid_size, pop, vacancy, tolerance)


In [ ]:
tolerance = .6

# first pick the number of frames. This is the same as the length of 
# the list of grids you are making.
n_frames = 40
# next make the list
grids = make_list(n_frames, (30, 30), .5, .1, tolerance)

#here we set up the initial colorbar figure
fig, ax = plt.subplots()
matrix = ax.matshow(grids[0], cmap='jet')
plt.colorbar(matrix)